In [1]:
%run ../spark-default.py

In [2]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [3]:
import builtins as b
from datetime import datetime, timedelta, date
import sys, traceback
from delta.tables import DeltaTable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# spark.sql("optimize stage.ciha_step1")

In [11]:
# Até aqui, a base do CIHA foi enriquecida com:
#  - Nome das cidades (residencia e movimentação)
#  - Nome, uf e info dos estados (residencia e movimentação)
#  - Infos do IBGE como população e área
#  - Calculo de feriados

In [4]:
ciha_step1 = spark.table("stage.ciha_step1")
ciha_step1.printSchema()

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- homonimo: string (nullable = true)
 |-- cnes:

In [5]:
ciha_step1

mes_cmpt,espec,cgc_hosp,munic_res,nasc,sexo,uti_mes_to,uti_int_to,proc_rea,qt_proc,dt_atend,dt_saida,diag_princ,diag_secun,cobranca,natureza,gestao,munic_mov,cod_idade,idade,dias_perm,morte,nacional,car_int,homonimo,cnes,fonte,cgc_consor,modalidade,input_file_name,arquivo_sigla_uf,ano_cmpt,munic_res_nome_uf,munic_mov_nome_uf,munic_res_nome_municipio,munic_mov_nome_municipio,munic_res_sigla_uf,munic_mov_sigla_uf,munic_res_regiao,munic_mov_regiao,munic_res_populacao_residente,munic_mov_populacao_residente,munic_res_area_unidade_territorial,munic_mov_area_unidade_territorial,feriado
03,null,61590410000124,355030,1948-06-29,3,0,002,0412040166,1,2022-07-27,2023-03-02,J90,J448,12,null,M,355030,4,74,218,0,null,null,null,2079127,01,1,02,s3a://datalake/so...,SP,2023,São Paulo,São Paulo,São Paulo,São Paulo,SP,SP,Sudeste,Sudeste,11253503,11253503,1521.101,1521.101,"{false, 1, [{2022..."
10,null,87547444000804,430210,1971-12-17,3,0,00,0302040013,1,2023-10-06,2023-10-06,Z501,null,18,null,M,430210,4,51,0,0,null,null,null,3392406,01,00000000000000,01,s3a://datalake/so...,RS,2023,Rio Grande do Sul,Rio Grande do Sul,Bento Gonçalves,Bento Gonçalves,RS,RS,Sul,Sul,107278,107278,381.958,381.958,"{false, 6, [{2023..."
02,null,00698882000100,354990,1979-08-31,3,0,0,0303170093,1,2022-11-10,2023-02-03,F29,null,12,null,M,354990,4,43,85,0,null,null,null,0009636,05,5,02,s3a://datalake/so...,SP,2023,São Paulo,São Paulo,São José dos Campos,São José dos Campos,SP,SP,Sudeste,Sudeste,629921,629921,1099.409,1099.409,"{false, 5, [{2022..."
10,null,87547444000804,430210,1954-10-19,1,0,00,0302050027,1,2023-10-06,2023-10-06,Z501,null,18,null,M,430210,4,68,0,0,null,null,null,3392406,01,00000000000000,01,s3a://datalake/so...,RS,2023,Rio Grande do Sul,Rio Grande do Sul,Bento Gonçalves,Bento Gonçalves,RS,RS,Sul,Sul,107278,107278,381.958,381.958,"{false, 6, [{2023..."
01,null,54012406000385,350330,1937-01-17,1,0,012,0301060010,1,2022-11-24,2023-01-07,I678,null,41,null,M,350330,4,85,44,1,null,null,null,2081040,01,00000000000000,02,s3a://datalake/so...,SP,2023,São Paulo,São Paulo,Araras,Araras,SP,SP,Sudeste,Sudeste,118843,118843,644.831,644.831,"{false, 4, [{2022..."
10,null,87547444000804,430210,1952-03-03,1,0,00,0302040013,1,2023-10-06,2023-10-06,Z501,null,18,null,M,430210,4,71,0,0,null,null,null,3392406,01,00000000000000,01,s3a://datalake/so...,RS,2023,Rio Grande do Sul,Rio Grande do Sul,Bento Gonçalves,Bento Gonçalves,RS,RS,Sul,Sul,107278,107278,381.958,381.958,"{false, 6, [{2023..."
02,null,00006037000470,355400,1946-07-23,1,0,041,0301060002,1,2022-12-06,2023-02-13,R418,null,12,null,M,355400,4,76,69,0,null,null,null,7368704,01,1,02,s3a://datalake/so...,SP,2023,São Paulo,São Paulo,Tatuí,Tatuí,SP,SP,Sudeste,Sudeste,107326,107326,523.475,523.475,"{false, 2, [{2022..."
10,null,87547444000804,430210,1957-04-06,3,0,00,0302040013,1,2023-10-06,2023-10-06,Z501,null,18,null,M,430210,4,66,0,0,null,null,null,3392406,01,00000000000000,01,s3a://datalake/so...,RS,2023,Rio Grande do Sul,Rio Grande do Sul,Bento Gonçalves,Bento Gonçalves,RS,RS,Sul,Sul,107278,107278,381.958,381.958,"{false, 6, [{2023..."
01,null,60922168005226,420240,1970-05-29,1,0,000,0305020056,1,2022-12-14,2023-01-03,N180,null,12,null,M,420240,4,52,20,0,null,null,null,2558246,01,1,02,s3a://datalake/so...,SC,2023,Santa Catarina,Santa Catarina,Blumenau,Blumenau,SC,SC,Sul,Sul,309011,309011,518.497,518.497,"{false, 6, [{2022..."
10,null,87547444000804,430210,2015-07-16,1,0,00,0302050027,1,2023-10-06,2023-10-06,Z501,null,18,null,M,430210,4,08,0,0,null,null,null,3392406,01,00000000000000,01,s3a://datalake/so...,RS,2023,Rio Grande do Sul,Rio Grande do Sul,Bento Gonçalves,Bento Gonçalves,RS,RS,Sul,Sul,107278,107278,381.958,381.958,"{false, 6, [{2023..."


In [6]:
cid10 = spark.table("stage.cid10_joined")
cid10.show(truncate=False)

+-------------+---------------------------+----------------+-----------+--------------------------------------------------------------------------------------------+-------------------------------------------------+------------------------+-----------------------+---------+-----------------------+------------------------------------------------------------------------------------------------+--------------------------------------------------+--------------------+---------------+------------------------------------------------------------+--------------------------------------------------+
|sub_categoria|sub_categoria_classificacao|restringido_sexo|causa_obito|sub_categoria_descricao                                                                     |sub_categoria_descricao_abreviada                |sub_categoria_referencia|sub_categoria_excluidos|categoria|categoria_classificacao|categoria_descricao                                                                             |categoria

In [7]:
ciha_step1.columns

['mes_cmpt',
 'espec',
 'cgc_hosp',
 'munic_res',
 'nasc',
 'sexo',
 'uti_mes_to',
 'uti_int_to',
 'proc_rea',
 'qt_proc',
 'dt_atend',
 'dt_saida',
 'diag_princ',
 'diag_secun',
 'cobranca',
 'natureza',
 'gestao',
 'munic_mov',
 'cod_idade',
 'idade',
 'dias_perm',
 'morte',
 'nacional',
 'car_int',
 'homonimo',
 'cnes',
 'fonte',
 'cgc_consor',
 'modalidade',
 'input_file_name',
 'arquivo_sigla_uf',
 'ano_cmpt',
 'munic_res_nome_uf',
 'munic_mov_nome_uf',
 'munic_res_nome_municipio',
 'munic_mov_nome_municipio',
 'munic_res_sigla_uf',
 'munic_mov_sigla_uf',
 'munic_res_regiao',
 'munic_mov_regiao',
 'munic_res_populacao_residente',
 'munic_mov_populacao_residente',
 'munic_res_area_unidade_territorial',
 'munic_mov_area_unidade_territorial',
 'feriado']

In [8]:
ciha_step2 = (ciha_step1.alias("s1")
    .join(cid10.alias("cid10_princ"),
          col("s1.diag_princ") == col("cid10_princ.sub_categoria"),
         "left")
    .join(cid10.alias("cid10_secun"),
          col("s1.diag_secun") == col("cid10_secun.sub_categoria"),
         "left")
    .select(
        col("s1.mes_cmpt"),
        col("s1.espec"),
        col("s1.cgc_hosp"),
        col("s1.munic_res"),
        col("s1.nasc"),
        col("s1.sexo"),
        col("s1.uti_mes_to"),
        col("s1.uti_int_to"),
        col("s1.proc_rea"),
        col("s1.qt_proc"),
        col("s1.dt_atend"),
        dayofweek('s1.dt_atend').alias("dt_atend_dow"),
        col("s1.dt_saida"),
        col("s1.diag_princ"),
        col("s1.diag_secun"),
        col("s1.cobranca"),
        col("s1.natureza"),
        col("s1.gestao"),
        col("s1.munic_mov"),
        col("s1.cod_idade"),
        col("s1.idade"),
        col("s1.dias_perm"),
        col("s1.morte"),
        col("s1.nacional"),
        col("s1.car_int"),
        col("s1.homonimo"),
        col("s1.cnes"),
        col("s1.fonte"),
        col("s1.cgc_consor"),
        col("s1.modalidade"),
        col("s1.input_file_name"),
        col("s1.arquivo_sigla_uf"),
        col("s1.ano_cmpt"),
        col("s1.munic_res_nome_uf"),
        col("s1.munic_mov_nome_uf"),
        col("s1.munic_res_nome_municipio"),
        col("s1.munic_mov_nome_municipio"),
        col("s1.munic_res_sigla_uf"),
        col("s1.munic_mov_sigla_uf"),
        col("s1.munic_res_regiao"),
        col("s1.munic_mov_regiao"),
        col("s1.munic_res_populacao_residente"),
        col("s1.munic_mov_populacao_residente"),
        col("s1.munic_res_area_unidade_territorial"),
        col("s1.munic_mov_area_unidade_territorial"),
        col("s1.feriado"),
        struct("cid10_princ.*").alias("diag_princ_cid10"),
        struct("cid10_secun.*").alias("diag_secun_cid10")
    )
)
    
ciha_step2.printSchema()
ciha_step2.show(truncate=False)

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_atend_dow: integer (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- 

In [9]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.ciha_step2") \
  .addColumns(ciha_step2.schema) \
  .partitionedBy("ano_cmpt") \
  .execute()

In [10]:
spark.sql("delete from stage.ciha_step2")

num_affected_rows
117092917


In [11]:
ciha_step2.write.mode("overwrite").insertInto("stage.ciha_step2")

In [12]:
spark.sql("optimize stage.ciha_step2")

path,metrics
s3a://datalake/st...,"{6, 12, {26794666..."


In [13]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [14]:
spark.sql("VACUUM stage.ciha_step2 RETAIN 0 HOURS")

path
s3a://datalake/st...


In [15]:
spark.stop()